# Distributed XGBoost on Dask in the cloud

This is the accompanying notebook to the blog post [XGBoost – frictionless training on datasets too big for the memory](https://coiled.io/blog/xgboost-frictionless-training/).

Swap in your dataset, spin up a cluster in 2 minutes and train at any scale!

### Cluster setup 

In [ ]:
from dask import dataframe as dd
import coiled

Order the cluster and look at it coming up in your [Coiled dashboard](https://cloud.coiled.io/):

In [ ]:
%%time
cluster = coiled.Cluster(n_workers=12, software="xgboost-on-coiled")

Connect to the cluster:

In [ ]:
from dask.distributed import Client, progress
client = Client(cluster)
client

### Load the dataset sample

In [ ]:
mortgage_data = dd.read_parquet(
    "s3://coiled-data/mortgage-2000.parq/*", 
    compression="gzip", 
    columns=columns, 
    storage_options={"anon":True}
)

mortgage_data

Pin the downloaded dataset to memory:

_This step reduces waiting times in subsequent steps that trigger computation._

In [ ]:
mortgage_data = mortgage_data.persist()

### Data preprocessing

The dataset needs a little work - we need to prepare categorical columns to a format that is supported by XGBoost.

The columns we'll be working with:

In [ ]:
columns = [
    "delinquency_12",
    "interest_rate",
    "loan_age",
    "adj_remaining_months_to_maturity",
    "longest_ever_deliquent",
    "orig_channel",
    "num_borrowers",
    "borrower_credit_score",
    "first_home_buyer",
    "loan_purpose",
    "property_type",
    "num_units",
    "occupancy_status",
    "property_state",
    "zip",
    "mortgage_insurance_percent",
    "coborrow_credit_score",
    "relocation_mortgage_indicator",
]
categorical = [
    "orig_channel",
    "occupancy_status",
    "property_state",
    "first_home_buyer",
    "loan_purpose",
    "property_type",
    "zip",
    "relocation_mortgage_indicator",
    "delinquency_12",
]

Create a column categorizer:

In [ ]:
from dask_ml.preprocessing import Categorizer

ce = Categorizer(columns=categorical)

Apply column categorizer:

In [ ]:
mortgage_data = ce.fit_transform(mortgage_data)

mortgage_data.dtypes

In [ ]:
# https://github.com/dmlc/xgboost/blame/9a0399e8981b2279d921fe2312f7ab1b880fd3c3/python-package/xgboost/dask.py#L227
# Dask categorical columns are not yet available

# the commit is already in master, can be expected in release 1.4.0

# Because this is not possible yet, I will cast to ints
for col in categorical:
    mortgage_data[col] = mortgage_data[col].cat.codes

### Split the dataset before training

In [ ]:
dependent_vars = mortgage_data.columns.difference(["delinquency_12"])
X, y = mortgage_data.iloc[:, dependent_vars], mortgage_data["delinquency_12"]

In [ ]:
from dask_ml.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, shuffle=True, random_state=2
)

### Train XGBoost

In [ ]:
import xgboost as xgb

Prepare distributed DMatrix structures: 

In [ ]:
dtrain = xgb.dask.DaskDMatrix(client, X_train, y_train)    
dtest = xgb.dask.DaskDMatrix(client, X_test, y_test)    

Training params:

In [ ]:
params = {
    "max_depth": 8,
    "max_leaves": 2 ** 8,
    "alpha": 0.9,
    "eta": 0.1,
    "gamma": 0.1,
    "learning_rate": 0.1,
    "subsample": 1,
    "reg_lambda": 1,
    "scale_pos_weight": 2,
    "min_child_weight": 30,
    "objective": "binary:logistic",
    "grow_policy": "lossguide",
}

Run training

In [ ]:
%%time
output = xgb.dask.train(
    client,
    params,
    dtrain,
    num_boost_round=20,
    evals=[
        (dtrain, 'train'), 
        (dtest, 'test')
    ]
)

Access results

In [ ]:
booster = output['booster']  # booster is the trained model
history = output['history']  # A dictionary containing evaluation 

In [ ]:
booster

In [ ]:
history

### Close session and set down the cluster

In [ ]:
cluster.close()

### Join our Slack community and share your success!

Follow this link to join:  
https://join.slack.com/t/coiled-users/shared_invite/zt-hx1fnr7k-In~Q8ui3XkQfvQon0yN5WQ

### Next steps

* Train on your own dataset
* Scale up the cluster to use more resources with `cluster.scale(24)`
* [GPU-accelerated XGBoost on Dask (NVidia RAPIDS team)](https://github.com/rapidsai-community/notebooks-contrib/blob/branch-0.14/intermediate_notebooks/E2E/mortgage/mortgage_e2e.ipynb)